# M1M3 cell learning
Craig Lage - 14-Apr-23 \
The 17 tons of mirror are supported by 156 pneumatic actuators where 44 are single-axis and provide support only on the axial direction, 100 are dual-axis providing support in the axial and lateral direction, and 12 are dual-axis providing support in the axial and cross lateral directions. \
Positioning is provided by 6 hard points in a hexapod configuration which moves the mirror to a fixed operational position that shall be maintained during telescope operations. The remaining optical elements will be moved relative to this position in order to align the telescope optics. Support and optical figure correction is provided by 112 dual axis and 44 single axis pneumatic actuators. 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LightSource as LS
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable

from lsst_efd_client import EfdClient



In [ ]:
client = EfdClient('summit_efd')
FATABLE = M1M3FATable.FATABLE

In [ ]:
# Times of bump test
start = Time("2023-04-19T12:52:00", scale='utc')
end = Time("2023-04-19T14:00:00", scale='utc')

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)

## Given an actuator ID, this plots the bump test result

In [ ]:
async def plotBumpTestResults(bumps, id):
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    # The pass/fail results are actually in the next test.
    lastThisBumpIndex = bumps[bumps['actuatorId']==id].last_valid_index()
    passFail = bumps.iloc[bumps.index.get_loc(lastThisBumpIndex)+1]
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    fig = plt.figure(figsize=(10,5))
    plt.subplots_adjust(wspace=0.3)
    plt.subplot(1,2,1)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 42.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    plotStart -= t0
    plotEnd -= t0
    plt.title(f"Primary - Z - ID:{id}")
    plt.plot(times, forces[primaryForce].values)
    if passFail[primaryBump] == 6:
        plt.text(2.0, 350.0, "PASSED", color='g')
    elif passFail[primaryBump] == 7:
        plt.text(2.0, 350.0, "FAILED", color='r')

    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.subplot(1,2,2)
    if secondaryName is not None:
        plt.title(f"Secondary - {secondaryName} - ID:{id}")
        plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 42.0 #thisBump[thisBump[secondaryBump]==5]['timestamp'].values[0] + 2.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        plotStart -= t0
        plotEnd -= t0
        plt.plot(times, forces[secondaryForce].values)
        if passFail[secondaryBump] == 6:
            plt.text(2.0, 350.0, "PASSED", color='g')
        elif passFail[secondaryBump] == 7:
            plt.text(2.0, 350.0, "FAILED", color='r')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
    else:
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
    return

In [ ]:
await plotBumpTestResults(bumps, 102)

## Now let's look at more of them

In [ ]:
# Times of bump test
start = Time("2020-06-01T00:00:00", scale='utc')
#end = Time("2022-06-01T00:00:00", scale='utc')
end = Time("2023-04-20T00:00:00", scale='utc')

In [ ]:
manyBumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)

In [ ]:
len(manyBumps)

In [ ]:
async def plotMultipleBumpTestResults(manyBumps, id):
    theseBumps = manyBumps[manyBumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    fig = plt.figure(figsize=(10,5))
    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Multiple bump tests Actuator ID {id}", fontsize=18)

    # Now find the separate tests
    times = theseBumps['timestamp'].values
    startTimes = []
    endTimes = []
    for i, time in enumerate(times):
        if i == 0:
            startTimes.append(time)
            continue
        if (time - times[i-1]) > 60.0:
            startTimes.append(time)
            endTimes.append(times[i-1])
    endTimes.append(times[-1])
    numPlots = 0
    for i in range(len(startTimes)):
        startTime = startTimes[i]
        endTime = endTimes[i]
        thisBump = theseBumps[(theseBumps['timestamp'] >= startTime) & (theseBumps['timestamp'] <= endTime)]
        
        try:
            numPlots += 1
            plt.subplot(1,2,1)
            plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
            plotEnd = plotStart + 14.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
            start = Time(plotStart, format='unix_tai', scale='tai')
            end = Time(plotEnd, format='unix_tai', scale='tai')
            forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
            times = forces['timestamp'].values
            t0 = times[0]
            times -= t0
            plotStart -= t0
            plotEnd -= t0
            lastIndex = bumps[bumps['actuatorId']==id].last_valid_index()
            passFail = bumps.iloc[bumps.index.get_loc(lastIndex)+1][primaryBump]
            if passFail == 6:
                print("PASSED")
            elif passFail == 7:
                print("FAILED")

            plt.title("Primary   Z")
            plt.plot(times, forces[primaryForce].values)
            plt.xlim(plotStart, plotEnd)
            plt.ylim(-400,400)
            plt.xlabel("Time (seconds)")
            plt.ylabel("Force (nt)")
            plt.subplot(1,2,2)
            if secondaryName is not None:
                plt.title(f"Secondary   {secondaryName}")
                plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
                plotEnd = plotStart + 14.0 #thisBump[thisBump[secondaryBump]==5]['timestamp'].values[0] + 2.0
                start = Time(plotStart, format='unix_tai', scale='tai')
                end = Time(plotEnd, format='unix_tai', scale='tai')
                forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
                times = forces['timestamp'].values
                t0 = times[0]
                times -= t0
                plotStart -= t0
                plotEnd -= t0
                plt.plot(times, forces[secondaryForce].values)
                plt.xlim(plotStart, plotEnd)
                plt.ylim(-400,400)
                plt.xlabel("Time (seconds)")
                plt.ylabel("Force (nt)")
            else:
                plt.title("No Secondary")
                plt.xticks([])
                plt.yticks([])
        except:
            continue
            
    plt.subplot(1,2,1)
    plt.text(2.0, 350, f"{numPlots} tests")
    plt.subplot(1,2,2)
    plt.text(2.0, 350, f"{numPlots} tests")
                
    #plt.savefig(f"/home/craiglagegit/DATA/M1M3_bump_tests/Bump_Tests_{id}.png")
    return

In [ ]:
await plotMultipleBumpTestResults(manyBumps, 115)

In [ ]:
id=227
thisBump = bumps[bumps['actuatorId']==id]
index = M1M3FATable.actuatorIDToIndex(id)
primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"

plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
plotEnd = plotStart + 14.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
start = Time(plotStart, format='unix_tai', scale='tai')
end = Time(plotEnd, format='unix_tai', scale='tai')
forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
times = forces['timestamp'].values
times -= times[0]
theseForces = forces[primaryForce].values

In [ ]:
from scipy.interpolate import UnivariateSpline
plt.plot(times, theseForces)
#spline = UnivariateSpline(times, theseForces, s=0.0)
plt.plot(times, spline(times))
rms = np.sqrt(np.mean((theseForces - spline(times))*(theseForces - spline(times))))
print(rms)

In [ ]:
len(spline(times))

In [ ]:
async def generateAverageBumpTest(manyBumps, id):
    theseBumps = manyBumps[manyBumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    # Now find the separate tests
    times = theseBumps['timestamp'].values
    startTimes = []
    endTimes = []
    for i, time in enumerate(times):
        if i == 0:
            startTimes.append(time)
            continue
        if (time - times[i-1]) > 60.0:
            startTimes.append(time)
            endTimes.append(times[i-1])
    endTimes.append(times[-1])
    numPlots = 0
    primarySplines = []
    secondarySplines = []
    for i in range(len(startTimes)):
        startTime = startTimes[i]
        endTime = endTimes[i]
        thisBump = theseBumps[(theseBumps['timestamp'] >= startTime) & (theseBumps['timestamp'] <= endTime)]
        # The pass/fail results are actually in the next test.
        lastThisBumpIndex = bumps[bumps['actuatorId']==id].last_valid_index()
        passFail = bumps.iloc[bumps.index.get_loc(lastThisBumpIndex)+1]
        if passFail[primaryBump] == 7:
            continue
        try:
            plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
            plotEnd = plotStart + 14.0
            start = Time(plotStart, format='unix_tai', scale='tai')
            end = Time(plotEnd, format='unix_tai', scale='tai')
            forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
            times = forces['timestamp'].values
            t0 = times[0]
            times -= t0
            primaryForces = forces[primaryForce].values
            primarySpline = UnivariateSpline(times, primaryForces, s=0.0)
            primarySplines.append(primarySpline)
            if secondaryName is not None:
                if passFail[secondaryBump] == 7:
                    continue
                plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
                plotEnd = plotStart + 14.0
                start = Time(plotStart, format='unix_tai', scale='tai')
                end = Time(plotEnd, format='unix_tai', scale='tai')
                forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
                times = forces['timestamp'].values
                t0 = times[0]
                times -= t0
                secondaryForces = forces[secondaryForce].values
                secondarySpline = UnivariateSpline(times, secondaryForces, s=0.0)
                secondarySplines.append(secondarySpline)
        except:
            continue

    return [primarySplines, secondarySplines]

In [ ]:
[primarySplines, secondarySplines] = await generateAverageBumpTest(manyBumps, 115)

In [ ]:
ts = np.linspace(0,14,5000)
fs = np.zeros_like(ts)
numSplines = len(primarySplines)
for spline in primarySplines:
    fs += spline(ts)
fs /= numSplines
averagePrimarySpline = UnivariateSpline(ts, fs)

In [ ]:
fs = np.zeros_like(ts)
numSplines = len(secondarySplines)
for spline in secondarySplines:
    fs += spline(ts)
fs /= numSplines
averageSecondarySpline = UnivariateSpline(ts, fs)

In [ ]:

plt.subplot(1,2,1)
plt.plot(ts, averagePrimarySpline(ts))
plt.subplot(1,2,2)
plt.plot(ts, averageSecondarySpline(ts))

In [ ]:
for spline in secondarySplines:
    plt.plot(ts, spline(ts))